In [ ]:
import sys
sys.path.append('..')
%load_ext autoreload
%autoreload 2


In [ ]:
import os
from pathlib import Path
import random

import matplotlib.pyplot as plt
import pickle
import torch
import numpy as np
from sklearn.model_selection import train_test_split

# from experiment_setup import get_model, set_random, build_estimator
from alpaca.model.ensemble import MLPEnsemble
from alpaca.dataloader.builder import build_dataset
from alpaca.uncertainty_estimator import build_estimator
from alpaca.analysis.metrics import get_uq_metrics
from experiments.utils.data import scale, multiple_kfold

plt.rcParams['figure.facecolor'] = 'white'

from catalyst.utils.seed import set_global_seed

In [ ]:
SEED = 10
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

torch.cuda.set_device(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False



In [ ]:
config = {
    'layers': [8, 128, 128, 64, 1],
    'n_ens': 2,
    'dropout_rate': 0.5,
    'dropout_uq': 0.5,
    'acc_percentile': .1
}

In [ ]:

# over datasets
dataset_name = 'boston_housing'
print(f'=={dataset_name}==')
dataset = build_dataset(dataset_name, val_split=0.0)
x_set, y_set = dataset.dataset('train')
config['layers'][0] = x_set.shape[-1]

x_train, x_val, y_train, y_val = train_test_split(x_set, y_set, train_size=0.2)
x_train, x_val, x_scaler = scale(x_train, x_val)
y_train, y_val, y_scaler = scale(y_train, y_val)

train_opts = {

}
# over model runs
ensemble = MLPEnsemble(config['layers'],
                    n_models=config['n_ens'],
                    reduction='mean')
ensemble.fit((x_train, y_train),
          (x_val, y_val),
          **train_opts)
model = ensemble.models[0]



In [ ]:
name = 'cov_leverages'
results = []
x_val_tensor = torch.DoubleTensor(x_val).cuda()
predictions = model(x_val_tensor).cpu().detach().numpy()

errors = np.abs(predictions - y_val)

unscale = lambda y : y_scaler.inverse_transform(y)
scaled_errors = unscale(predictions) - unscale(y_val)
rmse = np.sqrt(np.mean(np.square(scaled_errors)))

estimator = build_estimator(
    'mcdue_masked', model, nn_runs=100, dropout_mask=name, dropout_rate=config['dropout_uq'])

estimations = estimator.estimate(x_val_tensor)
acc, ndcg, ll = get_uq_metrics(estimations, errors,
                               config['acc_percentile'],
                               bins = [80, 95, 99]
                              )
results.append([acc, ndcg, ll, rmse, name ])
results[0][2]
